### Minimize the semanticKITTI dataset to only portion that has RGB color

Read the file that preprocess by preprocess_kitti (proj_h, proj_w, 10)

Cut the part where color is presented ()

clear data of any point that has color valid flag = 0 to undefined (=0)

clear data any point that has no range projection (=-1) to undefined (=0).

In [1]:
import numpy as np
import os
from tqdm.notebook import tqdm

In [2]:
sequences_rootdir = '..\\..\\sequences'
sequences = ['00', '01', '02', '03', '04', '05', '06', '07', '08', '09', '10']
# sequences = ['03']
proj_w=2048
proj_h=64

w_cut=[780,1260]
h_cut=[0,48]

In [3]:
for seq in tqdm(sequences, desc="Processing Sequences"):
    basedir = os.path.join(sequences_rootdir, seq)
    pointdir = os.path.join(sequences_rootdir, seq, 'preprocess')
    # search all .bin files in pointdir and put them in a list
    point_files = sorted([f for f in os.listdir(pointdir) if f.endswith('.bin')])

    # save the preprocess point cloud to a new file in the subfolder 'preprocess_mini', use the same filename
    preprocess_dir = os.path.join(basedir, 'preprocess_mini')
    os.makedirs(preprocess_dir, exist_ok=True)

    # go through the list of point files and image files and colorize the points
    for point_file in tqdm(point_files, total=len(point_files), desc="Colorizing point clouds", leave=False):
        pc_file = os.path.join(pointdir, point_file) 
        # This applied to semanticKITTI dataset only
        # Read the file into pcd and reshape as (proj_h, proj_w, 10)
        # pcd is the 2D numpy array with dimention (proj_h, proj_w, 10) and each elements include range, x, y, z, intensity, flag, r, g, b, label
        pcd = np.fromfile(pc_file, dtype=np.float32).reshape(proj_h, proj_w, 10)

        # Get region of interest using w_cut and h_cut
        roi = pcd[h_cut[0]:h_cut[1], w_cut[0]:w_cut[1], :]
        # clear data of any point that has flag = 0 (invalid color) or range = -1 (invalid range)
        mask_invalid = (roi[..., 5] == 0) | (roi[..., 0] == -1)
        roi[mask_invalid] = 0

        output_file = os.path.join(preprocess_dir, point_file)
        roi.astype(np.float32).tofile(output_file)


Processing Sequences:   0%|          | 0/11 [00:00<?, ?it/s]

Colorizing point clouds:   0%|          | 0/4541 [00:00<?, ?it/s]

Colorizing point clouds:   0%|          | 0/1101 [00:00<?, ?it/s]

Colorizing point clouds:   0%|          | 0/4661 [00:00<?, ?it/s]

Colorizing point clouds:   0%|          | 0/801 [00:00<?, ?it/s]

...

Colorizing point clouds:   0%|          | 0/1101 [00:00<?, ?it/s]

Colorizing point clouds:   0%|          | 0/1101 [00:00<?, ?it/s]

Colorizing point clouds:   0%|          | 0/4071 [00:00<?, ?it/s]

Colorizing point clouds:   0%|          | 0/1591 [00:00<?, ?it/s]

Colorizing point clouds:   0%|          | 0/1201 [00:00<?, ?it/s]

In [4]:
# proj_pointcloud is the 2D numpy array with dimention (proj_h, proj_w, 10) and each elements include range,x,y,z,intensity,flag,r,g,b,label
import matplotlib.pyplot as plt

# plot the range
def plot_range(proj_pointcloud):
    plt.imshow(proj_pointcloud[..., 0], cmap='jet')
    plt.axis('off') # do not show x and y axis
    plt.show()

# plot RGB image from proj_pointcloud
def plot_rgb(proj_pointcloud):
    # proj_pointcloud[..., 6:9] are r,g,b channels
    rgb = proj_pointcloud[..., 6:9]
    # Clip values to [0, 1] if normalized, else to [0, 255]
    if rgb.max() <= 1.0:
        rgb = (rgb * 255).astype(np.uint8)
    else:
        rgb = rgb.astype(np.uint8)
    plt.imshow(rgb)
    plt.axis('off')
    plt.show()

# plot label image from proj_pointcloud
def plot_label(proj_pointcloud):
    # proj_pointcloud[..., 9] is the label channel
    label_img = proj_pointcloud[..., 9]
    plt.imshow(label_img, cmap='tab20')
    plt.axis('off')
    plt.show()

# save range image
def save_range_image(proj_pointcloud, output_file):
    plt.imsave(output_file, proj_pointcloud[..., 0], cmap='jet')    
# save rgb image
def save_rgb_image(proj_pointcloud, output_file):
    rgb = proj_pointcloud[..., 6:9]
    if rgb.max() <= 1.0:
        rgb = (rgb * 255).astype(np.uint8)
    else:
        rgb = rgb.astype(np.uint8)
    plt.imsave(output_file, rgb)
# save label image
def save_label_image(proj_pointcloud, output_file):
    label_img = proj_pointcloud[..., 9]
    plt.imsave(output_file, label_img, cmap='tab20') 

In [5]:
save_range_image(roi, 'range_image.png')  # Save the range image
save_rgb_image(roi, 'rgb_image.png')  # Save the RGB image
save_label_image(roi, 'label_image.png')  # Save the label image